In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.append("../lib")

In [4]:
from bert_utils import Config, BertPreprocessor

In [5]:
config = Config(
    model_type="bert-base-uncased",
    max_seq_len=128,
)

In [6]:
processor = BertPreprocessor(config.model_type, config.max_seq_len)

In [7]:
from pytorch_pretrained_bert import BertConfig, BertForMaskedLM
# model = BertForMaskedLM.from_pretrained(config.model_type)
model = BertForMaskedLM.from_pretrained("../../../transformers/transformers/examples/language-modeling/output/")
model.eval() # https://jamesmccaffrey.wordpress.com/2019/01/23/pytorch-train-vs-eval-mode/
# need eval to turn off dropout?


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
   

In [8]:
def get_logits(sentence: str) -> np.ndarray:
    return model(processor.to_bert_model_input(sentence))[0, :, :].cpu().detach().numpy()

def softmax(arr, axis=1):
    e = np.exp(arr)
    return e / e.sum(axis=axis, keepdims=True)

from collections import defaultdict


# Parameters: sentence string, list of gender words
def get_mask_fill_logits(sentence: str, words: Iterable[str],
                         use_last_mask=False, apply_softmax=True) -> Dict[str, float]:
    mask_i = processor.get_index(sentence, "[MASK]", last=use_last_mask, accept_wordpiece=True)
    logits = defaultdict(list)
    out_logits = get_logits(sentence)
    if apply_softmax: 
        out_logits = softmax(out_logits)
    return {w: out_logits[mask_i, processor.token_to_index(w, accept_wordpiece=True)] for w in words}

def bias_score(sentence: str, gender_words: Iterable[Iterable[str]], 
               word: str, gender_comes_first=True) -> Dict[str, float]:
    """
    Input a sentence of the form "GGG is XXX"
    XXX is a placeholder for the target word
    GGG is a placeholder for the gendered words (the subject)
    We will predict the bias when filling in the gendered words and 
    filling in the target word.
    
    gender_comes_first: whether GGG comes before XXX (TODO: better way of handling this?)
    """
    # probability of filling [MASK] with "he" vs. "she" when target is "programmer"
    mwords, fwords = gender_words
    all_words = mwords + fwords
    subject_fill_logits = get_mask_fill_logits(
        sentence.replace("XXX", word).replace("GGG", "[MASK]"), 
        all_words, use_last_mask=not gender_comes_first,
    )
    subject_fill_bias = np.log(sum(subject_fill_logits[mw] for mw in mwords)) - \
                        np.log(sum(subject_fill_logits[fw] for fw in fwords))
    # male words are simply more likely than female words
    # correct for this by masking the target word and measuring the prior probabilities
    subject_fill_prior_logits = get_mask_fill_logits(
        sentence.replace("XXX", "[MASK]").replace("GGG", "[MASK]"), 
        all_words, use_last_mask=gender_comes_first,
    )
    subject_fill_bias_prior_correction = \
            np.log(sum(subject_fill_prior_logits[mw] for mw in mwords)) - \
            np.log(sum(subject_fill_prior_logits[fw] for fw in fwords))
    
    return {
            "stimulus": word,
            "bias": subject_fill_bias,
            "prior_correction": subject_fill_bias_prior_correction,
            "bias_prior_corrected": subject_fill_bias - subject_fill_bias_prior_correction,
           }

In [9]:
get_mask_fill_logits("the [MASK] is beautiful", ["flower", "bug"])

{'flower': 0.0006430532, 'bug': 1.6043728e-05}

In [10]:
def get_word_vector(sentence: str, word: str):
    idx = processor.get_index(sentence, word, accept_wordpiece=True)
    outputs = None
    with torch.no_grad():
        sequence_output, _ = model.bert(processor.to_bert_model_input(sentence),
                                        output_all_encoded_layers=False)
        sequence_output.squeeze_(0)
    return sequence_output.detach().cpu().numpy()[idx]

In [11]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [12]:
def get_effect_size(df1, df2, k="bias_prior_corrected"):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [13]:
def exact_mc_perm_test(xs, ys, nmc=100000):
    n, k = len(xs), 0
    diff = np.abs(np.mean(xs) - np.mean(ys))
    zs = np.concatenate([xs, ys])
    for j in range(nmc):
        np.random.shuffle(zs)
        k += diff < np.abs(np.mean(zs[:n]) - np.mean(zs[n:]))
    return k / nmc

In [14]:
get_word_vector("the flower is beautiful", "flower")

array([ 1.36417150e-01,  2.39513129e-01, -2.12734230e-02,  2.51169652e-01,
        4.10398304e-01, -1.57373801e-01, -7.34830201e-02,  7.45687485e-01,
        3.03634375e-01, -1.60973482e-02,  2.39932537e-03, -1.40317166e+00,
       -2.03687206e-01,  1.00155592e+00, -9.59882557e-01,  7.85475373e-01,
        1.13937914e-01,  1.08218038e+00,  1.22295544e-01,  9.11764801e-01,
       -1.92432031e-01, -5.38244724e-01,  3.84597555e-02, -6.20384395e-01,
        1.03891337e+00, -5.06965458e-01, -2.42486551e-01,  8.33453953e-01,
        4.23387080e-01,  9.96812060e-03,  1.84610203e-01,  9.12004560e-02,
        5.92462599e-01,  2.41602018e-01,  3.19918357e-02, -2.87897825e-01,
       -1.84869170e-01,  2.50599384e-01, -4.61469233e-01, -4.36794639e-01,
       -7.68599272e-01, -1.02913582e+00,  6.51616931e-01,  4.03380156e-01,
        1.92920774e-01, -7.00646341e-01, -1.02940345e+00,  3.47236395e-01,
        1.01606762e+00,  2.58834243e-01,  6.94141313e-02,  4.46886867e-01,
        1.81336731e-01, -

In [15]:
rev_vocab = {v:k for k, v in processor.full_vocab.items()}

In [16]:
from scipy.stats import ttest_ind, ranksums

In [17]:
from mlxtend.evaluate import permutation_test

# Flowers vs. Insects

All borrowed from WEAT

In [18]:
def to_words(wlist, filter_oov=True):
    return [w.strip() for w in wlist.lower().replace("\n", " ").split(", ") if w.strip() in rev_vocab or not filter_oov]

Words not in vocab are removed and target words are converted to adjectives when applicable and removed otherwise

In [19]:
# flower_words = to_words("""aster, clover, hyacinth, marigold, poppy, azalea, crocus, iris, orchid, rose, bluebell, daffodil, lilac, pansy, tulip, buttercup, daisy, lily, peony, violet, carnation, gladiola,
# magnolia, petunia, zinnia""")
# insect_words = to_words("""ant, caterpillar, flea, locust, spider, bedbug, centipede, fly, maggot, tarantula,
# bee, cockroach, gnat, mosquito, termite, beetle, cricket, hornet, moth, wasp, blackfly,
# dragonfly, horsefly, roach, weevil""")
flower_single_words = ["flower"]
flower_words = ["flowers"]
insect_single_words = ["bug"]
insect_words = ["bugs"]
pleasant_words = to_words("""caress, freedom, health, love, peace, cheer, friend, heaven, loyal, pleasure, diamond, gentle, honest, lucky, rainbow, diploma, gift, honor, miracle, sunrise, family,
happy, laughter, paradise, vacation""", filter_oov=False)
unpleasant_words = to_words("""abuse, crash, filth, murder, sickness, accident, death, grief, poison, stink,
assault, disaster, hatred, pollute, tragedy, divorce, jail, poverty, ugly, cancer, kill, rotten,
vomit, agony, prison""", filter_oov=False)

In [20]:
bias_score("GGG are XXX.", [flower_words, insect_words], "beautiful")

{'stimulus': 'beautiful',
 'bias': 5.1369779164502205,
 'prior_correction': 1.0994509323732693,
 'bias_prior_corrected': 4.037526984076951}

In [21]:
bias_score("GGG are XXX.", [flower_words, insect_words], "pleasant")

{'stimulus': 'pleasant',
 'bias': 3.7061896849218554,
 'prior_correction': 1.0994509323732693,
 'bias_prior_corrected': 2.606738752548586}

In [22]:
from itertools import product

In [23]:
df1 = pd.concat([
pd.DataFrame([bias_score("the GGG is XXX.", 
                         [flower_words, insect_words], w) for w in pleasant_words]),
pd.DataFrame([bias_score("GGG are XXX.", 
                         [flower_single_words, insect_single_words], w) for w in pleasant_words]),
])
df1

,stimulus,bias,prior_correction,bias_prior_corrected
0,caress,3.845614,2.944682,0.900933
1,freedom,1.451215,2.944682,-1.493467
2,health,0.283316,2.944682,-2.661365
3,love,2.636444,2.944682,-0.308237
4,peace,2.272286,2.944682,-0.672396
5,cheer,1.329683,2.944682,-1.614999
6,friend,2.100094,2.944682,-0.844587
7,heaven,2.763371,2.944682,-0.181310
8,loyal,-0.588951,2.944682,-3.533633
9,pleasure,1.866349,2.944682,-1.078332


In [24]:
df1["bias_prior_corrected"].mean()

0.06307908711837484

In [25]:
df2 = pd.concat([
pd.DataFrame([bias_score("the GGG is XXX.", 
                         [flower_words, insect_words], w) for w in unpleasant_words]),
pd.DataFrame([bias_score("GGG are XXX.", 
                         [flower_single_words, insect_single_words], w) for w in unpleasant_words]),
])
df2

,stimulus,bias,prior_correction,bias_prior_corrected
0,abuse,0.073523,2.944682,-2.871159
1,crash,0.141489,2.944682,-2.803192
2,filth,0.069981,2.944682,-2.874700
3,murder,-0.209847,2.944682,-3.154529
4,sickness,-0.920521,2.944682,-3.865202
5,accident,-0.012207,2.944682,-2.956889
6,death,0.374084,2.944682,-2.570598
7,grief,1.379701,2.944682,-1.564981
8,poison,0.567634,2.944682,-2.377047
9,stink,-0.460208,2.944682,-3.404889


In [26]:
df2["bias_prior_corrected"].mean()

-1.3768775163082199

Statistical test (is the t-test appropriate here?)

In [27]:
get_effect_size(df1, df2)

0.7071790335075726

In [28]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=3.7636634393334787, pvalue=0.0002853916830162073)

In [29]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=3.4400161178530992, pvalue=0.0005816795446878101)

In [30]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"], )

0.0003

### WEAT

In [31]:
wvs1 = [
    get_word_vector(f"[MASK] are {x}", x) for x in pleasant_words
]
wvs2 = [
    get_word_vector(f"[MASK] are {x}", x) for x in unpleasant_words
]

In [32]:
wv_insect = get_word_vector("insects are [MASK]", "insects")
sims_insect1 = [cosine_similarity(wv_insect, wv) for wv in wvs1]
sims_insect2 = [cosine_similarity(wv_insect, wv) for wv in wvs2]
mean_diff = np.mean(sims_insect1) - np.mean(sims_insect2)
std_ = np.std(sims_insect1 + sims_insect2)
effect_sz_insect = mean_diff / std_; effect_sz_insect

-0.12193329

In [33]:
wv_flower = get_word_vector("flowers are [MASK]", "flowers")
sims_flower1 = [cosine_similarity(wv_flower, wv) for wv in wvs1]
sims_flower2 = [cosine_similarity(wv_flower, wv) for wv in wvs2]
mean_diff = np.mean(sims_flower1) - np.mean(sims_flower2)
std_ = np.std(sims_flower1 + sims_flower2)
effect_sz_flower = mean_diff / std_; effect_sz_flower

0.17455073

In [34]:
exact_mc_perm_test(sims_insect1, sims_flower1)

0.0

In [35]:
exact_mc_perm_test(sims_insect2, sims_flower2)

1e-05

# Career vs Family

In [36]:
male_words = to_words("he")
female_words = to_words("she")
# male_words = to_words("John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill".lower())
# female_words = to_words("Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna".lower())
male_plural_words = to_words("boys, men")
female_plural_words = to_words("girls, women")
career_words = to_words("executive, management, professional, corporation, salary, office, business, career")
family_words = to_words("home, parents, children, family, cousins, marriage, wedding, relatives")

In [37]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in career_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in career_words]), 
])
df1

,stimulus,bias,prior_correction,bias_prior_corrected
0,executive,0.475477,0.410492,0.064985
1,management,0.506880,0.410492,0.096388
2,professional,0.632949,0.410492,0.222457
3,corporation,1.788665,0.410492,1.378173
4,salary,1.439565,0.410492,1.029073
5,office,0.581893,0.410492,0.171401
6,business,0.553903,0.410492,0.143411
7,career,1.091184,0.410492,0.680692
0,executive,0.005796,-0.360030,0.365826
1,management,0.123800,-0.360030,0.483830


In [38]:
df1["bias_prior_corrected"].mean()

0.39810667782064063

In [39]:
df2 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in family_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in family_words]), 
])
df2

,stimulus,bias,prior_correction,bias_prior_corrected
0,home,-0.242613,0.410492,-0.653105
1,parents,-0.153821,0.410492,-0.564313
2,children,-0.011720,0.410492,-0.422212
3,family,0.490244,0.410492,0.079752
4,cousins,0.284622,0.410492,-0.125870
5,marriage,0.056556,0.410492,-0.353936
6,wedding,0.074433,0.410492,-0.336059
7,relatives,0.175141,0.410492,-0.235351
0,home,-0.307303,-0.360030,0.052727
1,parents,-0.747479,-0.360030,-0.387449


In [40]:
df2["bias_prior_corrected"].mean()

-0.2910018063497258

Test for statistical significance

In [41]:
get_effect_size(df1, df2)

1.3826738703057957

In [42]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=6.622996664041908, pvalue=3.356218344768677e-08)

In [43]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=5.361109642475096, pvalue=8.271229396696217e-08)

In [44]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"], )

0.0

### WEAT

In [45]:
wvs1 = [
    get_word_vector(f"[MASK] like {x}", x) for x in family_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in family_words
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in family_words
]
wvs2 = [
    get_word_vector(f"[MASK] like {x}", x) for x in career_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in career_words    
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in career_words
]

In [46]:
wv_fm = get_word_vector("women like [MASK]", "women")
wv_fm2 = get_word_vector("she likes [MASK]", "she")
sims_fm1 = [cosine_similarity(wv_fm, wv) for wv in wvs1] +\
           [cosine_similarity(wv_fm, wv) for wv in wvs1]
sims_fm2 = [cosine_similarity(wv_fm, wv) for wv in wvs2] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_fm_family_career = mean_diff / std_; effect_sz_fm_family_career

1.570741

In [47]:
wv_m = get_word_vector("men like [MASK]", "men")
wv_m2 = get_word_vector("he likes [MASK]", "he")
sims_m1 = [cosine_similarity(wv_m, wv) for wv in wvs1]+\
           [cosine_similarity(wv_m2, wv) for wv in wvs1]
sims_m2 = [cosine_similarity(wv_m, wv) for wv in wvs2] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs2]
mean_diff = np.mean(sims_m1) - np.mean(sims_m2)
std_ = np.std(sims_m1 + sims_m1)
effect_sz_m_family_career = mean_diff / std_; effect_sz_m_family_career

0.2058285

In [48]:
exact_mc_perm_test(sims_fm1, sims_m1)

0.0003

In [49]:
exact_mc_perm_test(sims_fm2, sims_m2)

0.92416

# Math vs. Art

In [50]:
math_words = to_words("math, algebra, geometry, calculus, equations, computation, numbers, addition")
art_words = to_words("poetry, art, dance, Shakespear, literature, novels, symphony, drama, sculptures")

In [51]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in math_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in math_words]), 
])
df1

,stimulus,bias,prior_correction,bias_prior_corrected
0,math,0.276915,0.410492,-0.133577
1,algebra,0.455833,0.410492,0.045341
2,geometry,0.367912,0.410492,-0.042580
3,calculus,0.334713,0.410492,-0.075779
4,equations,0.844848,0.410492,0.434356
5,computation,0.921926,0.410492,0.511434
6,numbers,0.506278,0.410492,0.095786
7,addition,0.582013,0.410492,0.171521
0,math,-0.207428,-0.360030,0.152602
1,algebra,-0.564286,-0.360030,-0.204256


In [52]:
df1["bias"].mean()

0.45911435010955853

In [53]:
df2 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in art_words]),  
])
df2

,stimulus,bias,prior_correction,bias_prior_corrected
0,poetry,0.274590,0.410492,-0.135902
1,art,-0.072395,0.410492,-0.482887
2,dance,0.043486,0.410492,-0.367006
3,literature,0.404514,0.410492,-0.005978
4,novels,0.199505,0.410492,-0.210987
5,symphony,0.889284,0.410492,0.478792
6,drama,0.145099,0.410492,-0.265393
7,sculptures,0.321707,0.410492,-0.088785
0,poetry,-0.369240,-0.360030,-0.009210
1,art,-0.499251,-0.360030,-0.139221


In [54]:
df2["bias"].mean()

-0.024113005673514095

In [55]:
get_effect_size(df1, df2)

1.0939426757649762

In [56]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=4.4987571356002976, pvalue=4.62245822067383e-05)

In [57]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=3.958973274443148, pvalue=7.527265841737758e-05)

In [58]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

7e-05

### WEAT

In [59]:
wvs1 = [
    get_word_vector(f"[MASK] like {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in art_words
]
wvs2 = [
    get_word_vector(f"[MASK] like {x}", x) for x in math_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in math_words    
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in math_words
]

In [60]:
sims_fm1 = [cosine_similarity(wv_fm, wv) for wv in wvs1] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs1]
sims_fm2 = [cosine_similarity(wv_fm, wv) for wv in wvs2] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_fm_art_math = mean_diff / std_; effect_sz_fm_art_math

sims_m1 = [cosine_similarity(wv_m, wv) for wv in wvs1] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs1]
sims_m2 = [cosine_similarity(wv_m, wv) for wv in wvs2] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_m_art_math = mean_diff / std_; effect_sz_m_art_math

0.06520642

In [61]:
exact_mc_perm_test(sims_fm1, sims_m1)

0.7184

In [62]:
exact_mc_perm_test(sims_fm2, sims_m2)

0.96087

# Science vs. Art

In [63]:
science_words = to_words("science, technology, physics, chemistry, Einstein, NASA, experiments, astronomy")
art_words = to_words("poetry, art, dance, Shakespear, literature, novels, symphony, drama, sculptures")

In [64]:
df1 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in science_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in science_words]), 
])
df1

,stimulus,bias,prior_correction,bias_prior_corrected
0,science,0.584129,0.410492,0.173637
1,technology,0.711269,0.410492,0.300777
2,physics,0.472590,0.410492,0.062099
3,chemistry,0.085236,0.410492,-0.325256
4,einstein,0.272193,0.410492,-0.138299
5,nasa,0.871093,0.410492,0.460601
6,experiments,0.858208,0.410492,0.447716
7,astronomy,0.353417,0.410492,-0.057075
0,science,-0.108832,-0.360030,0.251198
1,technology,0.118942,-0.360030,0.478972


In [65]:
df1["bias"].mean()

0.5095245328430519

In [66]:
df2 = pd.concat([
    pd.DataFrame([bias_score("GGG likes XXX.", [male_words, female_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG like XXX.", [male_plural_words, female_plural_words], w) for w in art_words]),
    pd.DataFrame([bias_score("GGG is interested in XXX.", [["he"], ['she']], w) for w in art_words]), 
])
df2

,stimulus,bias,prior_correction,bias_prior_corrected
0,poetry,0.274590,0.410492,-0.135902
1,art,-0.072395,0.410492,-0.482887
2,dance,0.043486,0.410492,-0.367006
3,literature,0.404514,0.410492,-0.005978
4,novels,0.199505,0.410492,-0.210987
5,symphony,0.889284,0.410492,0.478792
6,drama,0.145099,0.410492,-0.265393
7,sculptures,0.321707,0.410492,-0.088785
0,poetry,-0.369240,-0.360030,-0.009210
1,art,-0.499251,-0.360030,-0.139221


In [67]:
df2["bias"].mean()

-0.024113005673514095

In [68]:
get_effect_size(df1, df2)

1.1761049873615947

In [69]:
ttest_ind(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

Ttest_indResult(statistic=5.011519116673934, pvalue=8.456927136719311e-06)

In [70]:
ranksums(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

RanksumsResult(statistic=4.062071536798438, pvalue=4.863916600395335e-05)

In [71]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

1e-05

### WEAT

In [72]:
wvs1 = [
    get_word_vector(f"[MASK] like {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in art_words
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in art_words
]
wvs2 = [
    get_word_vector(f"[MASK] like {x}", x) for x in science_words
] + [
    get_word_vector(f"[MASK] likes {x}", x) for x in science_words    
] + [
    get_word_vector(f"[MASK] is interested in {x}", x) for x in science_words
]

In [73]:
sims_fm1 = [cosine_similarity(wv_fm, wv) for wv in wvs1] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs1]
sims_fm2 = [cosine_similarity(wv_fm, wv) for wv in wvs2] +\
           [cosine_similarity(wv_fm2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_fm_art_math = mean_diff / std_; effect_sz_fm_art_math

sims_m1 = [cosine_similarity(wv_m, wv) for wv in wvs1] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs1]
sims_m2 = [cosine_similarity(wv_m, wv) for wv in wvs2] +\
           [cosine_similarity(wv_m2, wv) for wv in wvs2]
mean_diff = np.mean(sims_fm1) - np.mean(sims_fm2)
std_ = np.std(sims_fm1 + sims_fm1)
effect_sz_m_art_math = mean_diff / std_; effect_sz_m_art_math

0.18400623

In [74]:
exact_mc_perm_test(sims_fm1, sims_m1)

0.72035

In [75]:
exact_mc_perm_test(sims_fm2, sims_m2)

0.81264

# Black vs. White

In [76]:
df1 = pd.concat([
pd.DataFrame([bias_score("GGG people are XXX.", 
                         [["black"], ["white"]], w) for w in pleasant_words]),
pd.DataFrame([bias_score("the GGG person is XXX.", 
                         [["black"], ["white"]], w) for w in pleasant_words]),])
df1

,stimulus,bias,prior_correction,bias_prior_corrected
0,caress,-0.187789,-0.001936,-0.185853
1,freedom,0.044119,-0.001936,0.046055
2,health,-0.330293,-0.001936,-0.328357
3,love,-0.434663,-0.001936,-0.432726
4,peace,0.032176,-0.001936,0.034113
5,cheer,-0.177037,-0.001936,-0.175100
6,friend,-0.463606,-0.001936,-0.461669
7,heaven,-0.005531,-0.001936,-0.003595
8,loyal,0.048905,-0.001936,0.050841
9,pleasure,-0.589725,-0.001936,-0.587789


In [77]:
df2 = pd.concat([
pd.DataFrame([bias_score("GGG people are XXX.", 
                         [["black"], ["white"]], w) for w in unpleasant_words]),
pd.DataFrame([bias_score("the GGG person is XXX.", 
                         [["black"], ["white"]], w) for w in unpleasant_words]),
])
df2

,stimulus,bias,prior_correction,bias_prior_corrected
0,abuse,0.694155,-0.001936,0.696092
1,crash,0.434993,-0.001936,0.436929
2,filth,0.280639,-0.001936,0.282576
3,murder,0.541061,-0.001936,0.542997
4,sickness,-0.095774,-0.001936,-0.093837
5,accident,-0.200226,-0.001936,-0.198290
6,death,0.469533,-0.001936,0.471470
7,grief,0.526302,-0.001936,0.528239
8,poison,-0.009753,-0.001936,-0.007816
9,stink,0.843589,-0.001936,0.845525


In [78]:
get_effect_size(df1, df2)

-0.6661743317164707

In [79]:
exact_mc_perm_test(df1["bias_prior_corrected"], df2["bias_prior_corrected"])

0.00081

# Skills